In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim import models, similarities
from gensim.models import LdaModel, TfidfModel
from gensim.corpora import Dictionary
import time
import scipy as sp
import pickle
import sklearn as skl
from sklearn import feature_extraction
from sklearn import cluster
import spacy
from tqdm.notebook import tqdm

In [9]:
#recipe_df.to_pickle("src/data/pkls/recipe_ingrients_pp.pkl")
recipe_df = pickle.load(open("src/data/pkls/recipe_ingrients_pp.pkl","rb"))

In [39]:
count_vectorizer = feature_extraction.text.CountVectorizer()
documents = [" ".join(val) for val in recipe_df["pp_ingredients"].values]
cv_ingredients = count_vectorizer.fit_transform(documents)

In [4]:
pp_recipes = pickle.load(open("src/data/pkls/recipe_ingrients_pp.pkl","rb"))
ingr_corpus = pickle.load(open("src/data/pkls/ingr_corpus.pkl","rb"))
ingr_dict = pickle.load(open("src/data/pkls/ingr_dictionary.pkl","rb"))
index = similarities.SparseMatrixSimilarity(ingr_corpus, num_features = len(ingr_corpus))

In [5]:
def prep_ingr(ingredients):
    """preprocess formatting of the list of ingredients
    
    will remove string 'and' and '&' if present
    
    Args:
        ingredients (list of strings): list of ingredients
    
    Returns:
        list: list of formatted ingredients 
    """
    toreturn = []
    for ingr in ingredients:
        
        # remove 'and' or '&' if exsits
        if "and" in ingr or "&" in ingr:
            ingr = ingr.replace("and", "").replace("&","") #remove
            ingr = ingr.split(" ")
            # remove empty strings
            while "" in ingr:
                ingr.remove("")
                
            for i in ingr:
                toreturn.append(i)
        else:
            toreturn.append("_".join(ingr.split(" ")))
    return toreturn

In [44]:
def content_base_recommends(input_ingr,num_recs ,index = index, dct = ingr_dict, recipe_df = recipe_df):
    "returns certain amount of recipes that is similar to list of input ingredients"
    nlp = spacy.load("en_core_web_sm")
    ingrs = nlp(input_ingr)
    ingrs = " ".join([ingr.lemma_.lower()for ingr in ingrs])
    # format ingredients
    ingrs = prep_ingr(ingrs.split(" , "))
    
    # format ingredients in bow
    ingrs_bow = dct.doc2bow(ingrs)
    
    # get the n_closest recipes
    return recipe_df.iloc[index[ingrs_bow].argsort()[-num_recs:]]
    

In [70]:
my_ingr = "chicken breast, eggs, bacon, pepper , salt, Lettuce, onion, garlic"
content_base_recommends(my_ingr, 5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,ingredients_lemma,pp_ingredients
222776,venison maple bacon burgers,430939,11,1401545,2010-06-25,"[bacon, 15-minutes-or-less, time-to-make, cour...","[401.6, 43.0, 3.0, 23.0, 64.0, 47.0, 0.0]",5,"[put in food proscessor, mix togethar , so bac...",1 pound venison\r\n1/2 pound maple bacon\r\n1 ...,"[venison, bacon, onion, salt, pepper]",5,"venison , bacon , onion , salt , pepper","[venison, bacon, onion, salt, pepper]"
231384,zucchini squash and bacon saut,142858,40,53959,2005-10-26,"[bacon, 60-minutes-or-less, time-to-make, cour...","[575.1, 79.0, 26.0, 40.0, 33.0, 85.0, 4.0]",6,[cut bacon into approximately 1-inch size piec...,my grandmother used to make this as i was grow...,"[zucchini, bacon, onion, tomatoes, garlic, sal...",6,"zucchini , bacon , onion , tomato , garlic , s...","[zucchini, bacon, onion, tomato, garlic, salt,..."
40567,cheesy noodles and bacon,50256,20,52260,2003-01-07,"[30-minutes-or-less, time-to-make, course, mai...","[288.2, 35.0, 6.0, 26.0, 30.0, 50.0, 1.0]",11,[chop the onion and garlic to whatever size yo...,this is a simple recipe with a smoky flavor. i...,"[onion, bacon, garlic, egg, parmesan cheese, h...",9,"onion , bacon , garlic , egg , parmesan cheese...","[onion, bacon, garlic, egg, parmesan_cheese, h..."
97319,green beans with garlic,49824,40,65179,2003-01-03,"[60-minutes-or-less, time-to-make, course, mai...","[296.9, 26.0, 23.0, 14.0, 22.0, 28.0, 9.0]",4,"[cook bacon til just about crisp, add onions a...",this was the way my mother cooked green beans,"[green beans, garlic, bacon, onion, salt and p...",5,"green bean , garlic , bacon , onion , salt and...","[green_bean, garlic, bacon, onion, salt, pepper]"
151833,pammy s awful green beans w egg bacon onion,103213,10,166103,2004-11-02,"[15-minutes-or-less, time-to-make, course, mai...","[96.6, 8.0, 7.0, 15.0, 11.0, 8.0, 2.0]",8,"[spray a fry pan with cooking spray, using kit...",this is my version but my ex-mother-in-law use...,"[french style green beans, bacon, onion, egg, ...",5,"french style green bean , bacon , onion , egg ...","[french_style_green_bean, bacon, onion, egg, s..."


In [78]:
my_ingr = "chicken breast, eggs, bacon, pepper , salt, Lettuce, onion, garlic"
for recipes in content_base_recommends(my_ingr, 5).values:
    print(f"name: {recipes[0]}")
    print("recipe id: {}".format(recipes[1]))
    print()
    print(f"minutes: {recipes[2]}")
    print()
    print("ingredients:\n{}".format("\n".join(recipes[10])))
    print()
    print("steps:\n{}".format("\n".join(recipes[8])))
    print()
    print(f"description: {recipes[9]}")
    print("*"*10)

name: venison maple bacon burgers
recipe id: 430939

minutes: 11

ingredients:
venison
bacon
onion
salt
pepper

steps:
put in food proscessor
mix togethar , so bacon is mixed well with venison
make into burgers
and fry on grill or frying pan
salt and pepper

description: 1 pound venison
1/2 pound maple bacon
1 small onion

 it taste very good. i don't like venison. but i like it this way.i made this up.so i could eat the venison to.p.s. if you don't have that bacon, use maple sausge in place of maple bacon.
**********
name: zucchini squash and bacon saut
recipe id: 142858

minutes: 40

ingredients:
zucchini
bacon
onion
tomatoes
garlic
salt & pepper

steps:
cut bacon into approximately 1-inch size pieces and fry until crisp
add chopped onion and saut for a few minutes to lightly brown
add sliced zucchini , minced garlic , salt , pepper and stir
cover and lower heat
simmer on low heat , stirring occasionally until squash gets soft
add quartered tomatoes and simmer for 5-10 minutes more



NameError: name 'interaction' is not defined